In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/iris']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

## Keşifçi Veri Analizi | Becerileri Pekiştirme

Aşağıda ihtiyacımız doğrultusunda kullanacağımız kütüphaneleri yükleyelim.

In [1]:
import numpy as np
# FIRST-AUTHOR: remove plotting
# import seaborn as sns
# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])

Veri çerçevemizi bulunduğumuz dizinden yükleyelim ve bir veri çerçevesi haline getirerek df değişkenine atayalım. (pd.read_csv(...csv))

In [2]:
df = pd.read_csv("./input/iris.scaled.csv")

Veri çerçevesinin ilk 5 gözlemini görüntüleyelim.

In [3]:
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


Veri çerçevesinin kaç öznitelik ve kaç gözlemden oluştuğunu görüntüleyelim.

In [4]:
df.count()

sepal.length    150
sepal.width     150
petal.length    150
petal.width     150
variety         150
dtype: int64

Veri çerçevesindeki değişkenlerin hangi tipte olduğunu ve bellek kullanımını görüntüleyelim.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


Veri çerçevesindeki sayısal değişkenler için temel istatistik değerlerini görüntüleyelim.

Standart sapma ve ortalama değerlerden çıkarımda bulunarak hangi değişkenlerin ne kadar varyansa sahip olduğu hakkında fikir yürütelim.

In [6]:
# FIRST-AUTHOR: make notebook run
# df.mean()
df.mean(numeric_only=True)

sepal.length    5.843333
sepal.width     3.057333
petal.length    3.758000
petal.width     1.199333
dtype: float64

Veri çerçevesinde hangi öznitelikte kaç adet eksik değer olduğunu gözlemleyelim.

In [7]:
# FIRST-AUTHOR: make notebook run
# df.std()
df.std(numeric_only=True)

sepal.length    0.828066
sepal.width     0.435866
petal.length    1.765298
petal.width     0.762238
dtype: float64

Sayısal değişkenler arasında korelasyon olup olmadığını göstermek için korelasyon matrisi çizdirelim. Korelasyon katsayıları hakkında fikir yürütelim.

En güçlü pozitif ilişki hangi iki değişken arasındadır?

In [8]:
df.isna().sum()

sepal.length    0
sepal.width     0
petal.length    0
petal.width     0
variety         0
dtype: int64

Korelasyon katsayılarını daha iyi okuyabilmek için ısı haritası çizdirelim.

In [9]:
# FIRST-AUTHOR: make notebook run
# df.corr()
df.corr(numeric_only=True)

,sepal.length,sepal.width,petal.length,petal.width
sepal.length,1.000000,-0.117570,0.871754,0.817941
sepal.width,-0.117570,1.000000,-0.428440,-0.366126
petal.length,0.871754,-0.428440,1.000000,0.962865
petal.width,0.817941,-0.366126,0.962865,1.000000


Veri çerçevemizin hedef değişkeninin "variety" benzersiz değerlerini görüntüleyelim.

In [10]:
# FIRST-AUTHOR: make notebook run
# corr = df.corr()
corr = df.corr(numeric_only=True)
# FIRST-AUTHOR: remove plotting
# sns.heatmap(corr, 
#             xticklabels=corr.columns.values,
#             yticklabels=corr.columns.values);
_ = corr.columns.values
_ = corr.columns.values

Veri çerçevemizin hedef değişkeninin "variety" benzersiz kaç adet değer içerdiğini görüntüleyelim.

In [11]:
df["variety"].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

Veri çerçevesindeki sepal.width ve sepal.length değişkenlerinin sürekli olduğunu görüyoruz. Bu iki sürekli veriyi görselleştirmek için önce scatterplot kullanalım.

In [12]:
df["variety"].nunique()

3

Aynı iki veriyi daha farklı bir açıdan frekanslarıyla incelemek için jointplot kullanarak görselleştirelim. 

In [13]:
# FIRST-AUTHOR: remove plotting
# sns.scatterplot(x="sepal.width", y="sepal.length", data=df, color="magenta");

Aynı iki veriyi scatterplot ile tekrardan görselleştirelim fakat bu sefer "variety" parametresi ile hedef değişkenine göre kırdıralım. 

3 farklı renk arasında sepal değişkenleriyle bir kümeleme yapılabilir mi? Ne kadar ayırt edilebilir bunun üzerine düşünelim.

In [14]:
# FIRST-AUTHOR: remove plotting
# sns.jointplot(x="sepal.width", y="sepal.length", data=df, color="magenta");

value_counts() fonksiyonu ile veri çerçevemizin ne kadar dengeli dağıldığını sorgulayalım. 

In [15]:
# FIRST-AUTHOR: remove plotting
# sns.scatterplot(x="sepal.width", y="sepal.length", hue="variety", data=df);

Keman grafiği çizdirerek sepal.width değişkeninin dağılımını inceleyin. 

Söz konusu dağılım bizim için ne ifade ediyor, normal bir dağılım olduğunu söyleyebilir miyiz?

In [16]:
pd.value_counts(df.values.flatten())

/tmp/ipykernel_1156775/1709959354.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(df.values.flatten())


Versicolor    50
Setosa        50
Virginica     50
0.2           29
3.0           27
              ..
7.6            1
7.3            1
7.0            1
7.4            1
7.9            1
Name: count, Length: 77, dtype: int64

Daha iyi anlayabilmek için sepal.width üzerine bir distplot çizdirelim.

In [17]:
# FIRST-AUTHOR: remove plotting
# sns.violinplot(y = "sepal.width", data = df);

Üç çiçek türü için üç farklı keman grafiğini sepal.length değişkeninin dağılımı üzerine tek bir satır ile görselleştirelim.

In [18]:
# FIRST-AUTHOR: remove plotting
# sns.distplot(df["sepal.width"], bins=16, color="blue");
_ = df["sepal.width"]

Hangi çiçek türünden kaçar adet gözlem barındırıyor veri çerçevemiz?

50 x 3 olduğunu ve dengeli olduğunu value_counts ile zaten görmüştük, ancak bunu görsel olarak ifade etmek için sns.countplot() fonksiyonuna variety parametresini vereilm.

In [19]:
# FIRST-AUTHOR: remove plotting
# sns.violinplot(x = "variety", y = "sepal.length", data = df);

sepal.length ve sepal.width değişkenlerini sns.jointplot ile görselleştirelim, dağılımı ve dağılımın frekansı yüksek olduğu bölgelerini inceleyelim.

In [20]:
# FIRST-AUTHOR: remove plotting
# sns.countplot(x = "variety", data = df);

Bir önceki hücrede yapmış olduğumuz görselleştirmeye kind = "kde" parametresini ekleyelim. Böylelikle dağılımın noktalı gösterimden çıkıp yoğunluk odaklı bir görselleştirmeye dönüştüğünü görmüş olacağız.

In [21]:
# FIRST-AUTHOR: remove plotting
# sns.jointplot(x = df["sepal.width"], y = df["sepal.length"], color = "purple");
_ = df["sepal.width"]
_ = df["sepal.length"]

scatterplot ile petal.length ve petal.width değişkenlerinin dağılımlarını çizdirelim.

In [22]:
# FIRST-AUTHOR: remove plotting
# sns.jointplot(x = df["sepal.width"], y = df["sepal.length"],kind="kde", color = "purple");
_ = df["sepal.width"]
_ = df["sepal.length"]

Aynı görselleştirmeye hue = "variety" parametresini ekleyerek 3. bir boyut verelim.

In [23]:
# FIRST-AUTHOR: remove plotting
# sns.scatterplot(x="petal.width", y="petal.length", data=df);

sns.lmplot() görselleştirmesini petal.length ve petal.width değişkenleriyle implemente edelim. Petal length ile petal width arasında ne tür bir ilişki var ve bu ilişki güçlü müdür? sorusunu yanıtlayalım.

In [24]:
# FIRST-AUTHOR: remove plotting
# sns.scatterplot(x="petal.width", y="petal.length",hue="variety", data=df);

Bu sorunun yanıtını pekiştirmek için iki değişken arasında korelasyon katsayısını yazdıralım. 

In [25]:
# FIRST-AUTHOR: remove plotting
# sns.lmplot(x="petal.width", y="petal.length", data=df);

Petal Length ile Sepal Length değerlerini toplayarak yeni bir total length özniteliği oluşturalım.

In [26]:
df[['petal.length', 'petal.width']].corr()

,petal.length,petal.width
petal.length,1.000000,0.962865
petal.width,0.962865,1.000000


total.length'in ortalama değerini yazdıralım. 

In [27]:
df['total.length'] = df['petal.length'] + df['sepal.length']

total.length'in standart sapma değerini yazdıralım.

In [28]:
df['total.length'].mean()

9.601333333333333

sepal.length'in maksimum değerini yazdıralım.

In [29]:
df['total.length'].std()

2.5200401260965

sepal.length'i 5.5'den büyük ve türü setosa olan gözlemleri yazdıralım.

In [30]:
df['sepal.length'].max()

7.9

petal.length'i 5'den küçük ve türü virginica olan gözlemlerin sadece sepal.length ve sepal.width değişkenlerini ve değerlerini yazdıralım.

In [31]:
df[(df['variety']=='Setosa') & (df['sepal.length']>5.5)]

,sepal.length,sepal.width,petal.length,petal.width,variety,total.length
14,5.8,4.0,1.2,0.2,Setosa,7.0
15,5.7,4.4,1.5,0.4,Setosa,7.2
18,5.7,3.8,1.7,0.3,Setosa,7.4


Hedef değişkenimiz variety'e göre bir gruplama işlemi yapalım değişken değerlerimizin ortalamasını görüntüleyelim.

In [32]:
df[(df['variety']=='Virginica') & (df['petal.length']<5)][['sepal.length','sepal.width']]

,sepal.length,sepal.width
106,4.9,2.5
121,5.6,2.8
123,6.3,2.7
126,6.2,2.8
127,6.1,3.0
138,6.0,3.0


Hedef değişkenimiz variety'e göre gruplama işlemi yaparak sadece petal.length değişkenimizin standart sapma değerlerini yazdıralım. 

In [33]:
df.groupby(["variety"]).mean()

,sepal.length,sepal.width,petal.length,petal.width,total.length
variety,,,,,
Setosa,5.006,3.428,1.462,0.246,6.468
Versicolor,5.936,2.770,4.260,1.326,10.196
Virginica,6.588,2.974,5.552,2.026,12.140


Emeğiniz, ayırdığınız vakit ve ilginiz için teşekkürler.